In [1]:
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

In [2]:
import pandas as pd
import os

In [16]:
# Load datasets
datadir = '../data/'
os.makedirs(datadir, exist_ok=True)
datasets = ['azathoth42', 'svanoo']
substrings = ['myanimelist', 'myanimelist-dataset']
for ds_name, ss in zip(datasets, substrings):
    if not f'{ds_name}' in os.listdir(datadir):
        os.makedirs(f'{datadir}/{ds_name}', exist_ok=True)
        api.dataset_download_files(f'{ds_name}/{ss}', path=os.path.join(datadir, ds_name), unzip=True)

In [8]:
from pprint import pprint


chunksize = 10000
headsize = 10
table_names = os.listdir(datadir )
table_names = [
    tname for tname in table_names if os.path.splitext(tname)[-1] == '.csv']
table_dict = {}
log_file = os.path.join(datadir, f'log_head{headsize}.txt')
if os.path.exists(log_file):
    os.remove(log_file)
for tname in table_names:
    tpath = os.path.join(datadir, tname)
    if 'cleaned' in tname:
        os.replace(tpath, os.path.join(datadir, 'cleaned', tname))
    elif 'filtered' in tname:
        os.replace(tpath, os.path.join(datadir, 'filtered', tname))
    else:
        if chunksize:
            tfr_iter = pd.read_csv(tpath,
                                   chunksize=chunksize,
                                   on_bad_lines='skip')
            for chunk in tfr_iter:
                print(tname)
                print(chunk.columns)
                print('\n')
                table_dict[os.path.splitext(tname)[0]] = chunk
                with open(os.path.join(datadir, f'log_head{headsize}.txt'), 'a') as f:
                    f.write(tname)
                    f.write('\n')
                    f.write(chunk.head(headsize).to_string())
                    f.write('\n\n')
                break
        else:
            tbl = pd.read_csv(tpath,
                              chunksize=chunksize,
                              on_bad_lines='skip')
            print(tname)
            print(tbl.columns)
            print('\n')
            table_dict[os.path.splitext(tname)[0]] = tbl
            with open(os.path.join(datadir, f'log_head{headsize}.txt'), 'a') as f:
                f.write(tname)
                f.write('\n')
                f.write(tbl.head(headsize).to_string())
                f.write('\n\n')

pprint(table_dict.keys())

AnimeList.csv
Index(['anime_id', 'title', 'title_english', 'title_japanese',
       'title_synonyms', 'image_url', 'type', 'source', 'episodes', 'status',
       'airing', 'aired_string', 'aired', 'duration', 'rating', 'score',
       'scored_by', 'rank', 'popularity', 'members', 'favorites', 'background',
       'premiered', 'broadcast', 'related', 'producer', 'licensor', 'studio',
       'genre', 'opening_theme', 'ending_theme'],
      dtype='object')


UserAnimeList.csv
Index(['username', 'anime_id', 'my_watched_episodes', 'my_start_date',
       'my_finish_date', 'my_score', 'my_status', 'my_rewatching',
       'my_rewatching_ep', 'my_last_updated', 'my_tags'],
      dtype='object')


UserList.csv
Index(['username', 'user_id', 'user_watching', 'user_completed', 'user_onhold',
       'user_dropped', 'user_plantowatch', 'user_days_spent_watching',
       'gender', 'location', 'birth_date', 'access_rank', 'join_date',
       'last_online', 'stats_mean_score', 'stats_rewatched', 'stats

In [10]:
animelist_df = table_dict['AnimeList']
allgenres = set(animelist_df['genre'].fillna('NONE').str.cat(sep=', ').split(', '))
print(len(allgenres))

44


In [45]:
userlist = pd.read_csv('../data/svanoo/user.csv', delimiter='\t')
userlist.rename(columns={'user_id': 'user_name'}, inplace=True)
userlist = userlist.rename_axis('user_id').reset_index()
userlist.sort_values(by=['last_online_date'], ascending=False).head()

,user_id,user_name,user_url,last_online_date,num_watching,num_completed,num_on_hold,num_dropped,num_plan_to_watch,num_days,mean_score,clubs
696328,696328,literaturenerd,https://myanimelist.net/profile/literaturenerd,2022-03-20 04:25:30,5,1202,0,20,11,250.6,4.66,67029|76828|74845|72667|67302|17705|68087|2624...
753950,753950,nishant0,https://myanimelist.net/profile/nishant0,2022-03-20 03:46:43,6,190,7,14,70,59.9,6.80,67199|84113|32683
895383,895383,paladin23,https://myanimelist.net/profile/Paladin23,2022-03-20 02:02:31,10,614,0,0,118,110.4,7.48,807|63421|76804|27137
662762,662762,fury494,https://myanimelist.net/profile/fury494,2022-03-20 01:47:06,5,162,1,1,27,40.0,7.88,NaN
447110,447110,sleepingbear,https://myanimelist.net/profile/SleepingBear,2022-03-20 01:38:48,3,319,33,37,215,63.3,6.53,70446|12591|149|72408|67199|72490|70059|18611


In [43]:
userlist.describe()

,num_watching,num_completed,num_on_hold,num_dropped,num_plan_to_watch,num_days,mean_score,user_id
count,1.123284e+06,1.123284e+06,1.123284e+06,1.123284e+06,1.123284e+06,1.123284e+06,1.123284e+06,1.123284e+06
mean,1.148724e+01,1.590848e+02,7.178382e+00,9.060373e+00,6.562002e+01,5.573767e+01,7.122095e+00,5.616415e+05
std,3.529409e+01,2.287495e+02,2.646056e+01,4.703697e+01,1.590595e+02,1.344330e+02,2.477115e+00,3.242643e+05
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.000000e+00,3.700000e+01,0.000000e+00,0.000000e+00,3.000000e+00,1.600000e+01,7.050000e+00,2.808208e+05
50%,5.000000e+00,9.700000e+01,1.000000e+00,2.000000e+00,2.100000e+01,3.860000e+01,7.730000e+00,5.616415e+05
75%,1.100000e+01,2.020000e+02,6.000000e+00,8.000000e+00,7.000000e+01,7.370000e+01,8.350000e+00,8.424622e+05
max,1.101400e+04,1.865900e+04,6.933000e+03,1.625000e+04,1.864000e+04,1.053386e+05,1.000000e+01,1.123283e+06


In [44]:
userlist[userlist['user_url'] == 'https://myanimelist.net/profile/mortarSynth']

,user_name,user_url,last_online_date,num_watching,num_completed,num_on_hold,num_dropped,num_plan_to_watch,num_days,mean_score,clubs,user_id
219286,mortarsynth,https://myanimelist.net/profile/mortarSynth,2021-10-31 16:44:00,0,207,4,6,14,58.6,7.88,NaN,219286
